# 18-1. Project: 멋진 챗봇 만들기

## 라이브러리 버전을 확인해 봅니다

---

사용할 라이브러리 버전을 둘러봅시다

In [1]:
import numpy 
import pandas 
import tensorflow 
import nltk
import gensim

print(numpy.__version__)
print(pandas.__version__)
print(tensorflow.__version__)
print(nltk.__version__)
print(gensim.__version__)

1.21.4
1.3.3
2.6.0
3.6.5
4.1.2


## Step 1. 데이터 다운로드

---

`ChatbotData .csv`를 `pandas` 라이브러리로 읽어서 데이터의 질문과 답변을 각각 `questions`, `answers` 변수에 나눠서 저장하기  
- [songys/Chatbot_data](https://github.com/songys/Chatbot_data)

In [2]:
# 사용할 모듈들 선언
import pandas as pd
import tensorflow as tf
import numpy as np
from konlpy.tag import Mecab
import re
import os
import random
import matplotlib.pyplot as plt

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

*파일 경로 선언*

In [3]:
file_dir = '/aiffel/aiffel/transformer_chatbot/data'
file_name = 'ChatbotData.csv'
file_path = '/'.join([file_dir, file_name])

print(file_path)

/aiffel/aiffel/transformer_chatbot/data/ChatbotData.csv


*CSV 읽고 데이터 확인하기*

In [4]:
dataset = pd.read_csv(file_path)
dataset.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [5]:
# 데이터 질문 저장
questions = [q for q in dataset['Q']]
# 데이터 답변 저장
answers = [a for a in dataset['A']]

*저장된 질문, 답변 확인*

In [6]:
questions[:5]

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']

In [7]:
len(questions)

11823

In [8]:
answers[:5]

['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.']

In [9]:
len(answers)

11823

## Step 2. 데이터 정제

---

아래 조건을 만족하는 `preprocess_sentence()` 함수 구현하기  
1. 영문자의 경우, **모두 소문자로 변환**
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 **정규식을 활용하여 모두 제거**

In [10]:
def preprocess_sentence(sentence):
    # 1. 영문자의 경우, 모두 소문자로 변환
    sentence = sentence.lower()
    # 2. 정규식을 활용하여 조건에 해당하는 것들을 제거
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣!?])', " ", sentence)
    
    return sentence

## Step 3. 데이터 토큰화

---

토큰화 &rarr; KoNLPy의 `mecab` 클래스 사용  
아래 조건을 만족하는 `build_corpus()` 함수 구현하기  
1. **소스 문장 데이터, 타켓 문장 데이터** 입력으로 받기
2. 데이터를 `preprocess_sentece()`로 **정제하고, 토큰화**
3. 토큰화 &rarr; **전달 받은 토크나이즈 함수 사용** (`mecab.morphs`)
4. 토큰의 개수가 일정 길이 이상인 문장은 **데이터에서 제외**
5. **중복되는 문장은 데이터에서 제외**
    - `소스`는 소스대로, `타겟`은 타겟대로 검사, 중복 쌍이 흐트러지지 않도록 주의

In [11]:
# 함수구현; 소스 문장, 타겟 문장 데이터들 입력으로 받기
## 소스 --> 질문(question), 타켓 --> 답변(answer)
def build_corpus(src, tgt):
    # 토큰 길이 제한 값 (암의설정)
    MAX_LEN = 50
    
    # 데이터 정제
    src = preprocess_sentence(src)
    tgt = preprocess_sentence(tgt)
    # 토큰화
    mecab = Mecab() # Mecab 인스턴스 생성
    ## 토큰의 개수가 일정 길이(MAX_LEN) 이상인 문장은 제외
    if MAX_LEN < len(mecab.morphs(src)):
        src_corpus = mecab.morphs(src)
        # 중복되는 문장은 데이터에서 제외
        src_corpus = list(set(src_corpus))
    if MAX_LEN < len(mecab.morphs(tgt)):
        tgt_corpus = mecab.morphs(tgt)
        # 중복되는 문장은 데이터에서 제외
        tgt_corpus = list(set(tgt_corpus))
    
    # 각 데이터별로 검사 후 중복 쌍이 흐트러지지 않도록 List로 감싸서 return
    return [src_corpus, tgt_corpus]

## Step 4. Augmentation

---

주어진 데이터: **약 1만 개** (데이터가 적으므로 **Lexical Substitution 적용**)  
아래 링크 참고해 **한국어로 사전 훈련된 Embedding 모델 다운로드**.  
`Korean (w)`를 사이트에서 찾아 다운로드하고, `ko.bin` 파일을 얻기

- [Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)

다운로드한 모델 활용해 **데이터 Augmentation** (앞서 정의한 `lexical_sub()` 함수 참고)  
1. `que_corpus`: Augmentation 된 말뭉치
2. `ans_corpus`: 원본 말뭉치

---

3. `que_corpus`: 원본 말뭉치
4. `ans_corpus`: Augmentation 된 말뭉치

> `1:2`, `3:4`가 병렬을 이루도록하여 **전체 데이터가 원래의 3배 가량으로 늘어나도록 조치**


- 위 링크에서 필요한 파일들 다운로드 완료 (local)
- 이후 LMS 업로드 완료 (`*.bin`, `*.tsv`)
- 파일 경로 (디렉토리 까지만 표시함)
    > `/aiffel/aiffel/transformer_chatbot/data/Kyubyong_wordvectors`

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
#### 사전 학습된 모델을 가지고 텍스트를 Augmentation (일부 단어 대체)하는 함수 선언

# 모델 경로 및 토크나이저 선택
pre_trained_model_path = '/aiffel/aiffel/transformer_chatbot/data/Kyubyong_wordvectors/ko.bin'
pre_trained_tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")

# 모델 로드
pre_trained_model = AutoModelForMaskedLM.from_pretrained(pre_trained_model_path)

# lexical_sub() 함수 정의
def lexical_substitution(text, model, tokenizer):
    # 입력 텍스트를 토큰화
    tokens = tokenizer.tokenize(text)
    
    # 무작위로 대체할 토큰을 선택
    token_to_replace = random.choice(tokens)
    
    # 선택한 토큰을 '[MASK]'로 대체
    tokens[tokens.index(token_to_replace)] = '[MASK]'
    
    # 토큰을 모델 입력 형식으로 변환
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    # 모델에 입력을 전달하여 대체 단어 예측
    with tf.device('/GPU:0'):
        input_tensor = tf.constant([input_ids])
        outputs = model(input_tensor)
        predictions = outputs.logits[0]
    
    # 각 [MASK] 위치에서 가장 유사한 단어로 대체
    for i, token_id in enumerate(input_ids):
        if token_id == tokenizer.mask_token_id:
            predicted_index = np.argmax(predictions[i])
            predicted_word = tokenizer.convert_ids_to_tokens([predicted_index])[0]
            tokens[i] = predicted_word
    
    # 대체된 토큰을 문장으로 복원
    augmented_text = tokenizer.convert_tokens_to_string(tokens)
    
    return augmented_text

In [ ]:
# 원본 questions를 돌면서 Augmentation 수행해 text 내용 일부 변경하기
augmented_questions = []

for sentence in questions:
    augmented_questions.append()

## Step 5. 데이터 벡터화

---

`ans_corpus`: 타겟 데이터 (`<start>`, `<end>` 토큰을 추가해주고 벡터화 진행하기)

```python
sample_data = ["12", "시", "땡", "!"]

print(["<start>"] + sample_data + ["<end>"])
```
```shell
['<start>', '12', '시', '땡', '!', '<end>']
```

1. 위 소스 참고하여 타겟 데이터 전체에 토큰 추가 (`<start>`, `<end>`)
2. 특수 토큰이 더해진 `ans_corpus`에 `que_corpus`를 결합하여 **전체 데이터에 대한 단어 사전을 구축하고 벡터화**하여 `enc_train`, `dec_train` 얻기

## Step 6. 훈련하기

---

앞서 훈련한 모델 `Transformer`를 그대로 사용하기  
대신 데이터 크기 작으니 하이퍼파라미터를 튜닝하여 과적합 피하기  
모델 훈련하고 아래 예문에 대한 답변을 생성하기

```shell
# 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.

---

# 제출

Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>

Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2

Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10
```

## Step 7. 성능 측정하기

---

1. 주어진 질문에 적절한 답변을 하는지 확인
2. BLEU Score를 계산하는 `calculate_bleu()` 함수도 적용해보기

### 루브릭
#### 아래의 기준을 바탕으로 프로젝트를 평가합니다.

|**평가문항**|**상세기준**|
|---|---|
|1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?|챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.|
|2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?|과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.|
|3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?|주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.|